# Deploying Machine Learning Models Minikube with RBAC using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models. It uses Minikube as the target Kubernetes cluster.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)


# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

**2018-06-13** : At present we find the most stable version of minikube across platforms is 0.25.2 as there are issues with 0.26 and 0.27 on some systems. We also find the default VirtualBox driver can be problematic on some systems to we suggest using the [KVM2 driver](https://github.com/kubernetes/minikube/blob/master/docs/drivers.md#kvm2-driver).

Your start command would then look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [1]:
!kubectl create namespace seldon

namespace "seldon" created


In [2]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding "kube-system-cluster-admin" created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

deployment "tiller-deploy" successfully rolled out


Label the node to allow load testing to run on it

In [6]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "minikube" labeled


## Start seldon-core

Install the custom resource definition

In [7]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Sep 12 17:14:54 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s


NOTES:
NOTES: TODO




In [8]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Wed Sep 12 17:14:56 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1beta1/RoleBinding
NAME        AGE
ambassador  1s

==> v1/RoleBinding
seldon  1s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador-admin  NodePort   10.96.90.209    <none>       8877:31017/TCP                 1s
seldon-core-ambassador        NodePort   10.103.21.21    <none>       8080:31688/TCP                 1s
seldon-core-seldon-apiserver  NodePort   10.107.181.242  <none>       8080:32704/TCP,5000:31255/TCP  1s
seldon-core-redis             ClusterIP  10.104.240.70   <none>       6379/TCP                       1s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  1s

==> v1beta1/Role
seldon-local  1s
ambassador    1s

==> v1beta1/Deployment
NAME                    

Install prometheus and grafana for analytics

In [9]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

NAME:   seldon-core-analytics
LAST DEPLOYED: Wed Sep 12 17:15:00 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          0s
grafana-prom-deployment  1        1        1           0          0s
prometheus-deployment    1        1        1           0          0s

==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <none>         0s

==> v1/Pod(related)
NAME                                      READY  STATUS             RESTARTS  AGE
grafana-prom-import-dashboards-4kck6      0/1    ContainerCreating  0         0s
alertmanager-deployment-7fbfdfdfb6-89f99  0/1    ContainerCreating  0         0s
grafana-prom-deployment-7b45fb85d4-6zr8h  0/1    ContainerCreating  0         0s
prometheus-node-exporter-qgfv

In [10]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
Waiting for rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:8080
```

Install gRPC modules for the prediction protos.

In [11]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

In [12]:
from seldon_utils import *
API_AMBASSADOR="localhost:8004"

Illustration of both REST and gRPC requests. 

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [13]:
!kubectl create -f resources/model_invalid1.json -n seldon

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"apiVersion":"machinelearning.seldon.io/v1alpha2", "kind":"SeldonDeployment", "metadata":map[string]interface {}{"labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "namespace":"seldon", "creationTimestamp":"2018-09-12T16:26:46Z", "uid":"a44bb2ec-b6a8-11e8-8e6f-107a03897591", "selfLink":"", "clusterName":""}, "spec":map[string]interface {}{"annotations":map[string]interface {}{"deployment_version":"v1", "project_name":"FX Market Prediction"}, "name":"test-deployment", "oauth_key":1234, "oauth_secret":"oauth-secret", "predictors":[]interface {}{map[string]interface {}{"componentSpecs":[]interface {}{map[string]interface {}{"spec":map[string]interface {}{"containers":[]interface {}{map[string]interface {}{"image":"seldonio/mean_classifier:0.6", "imagePullPolicy":22, "name":"mean-classifier", "resources":map[string]interface {}{"requests":map[string]int

## Normal Operation
A simple example is shown below we use a single prepacked model for illustration. The spec contains a set of predictors each of which contains a ***componentSpec*** which is a Kubernetes [PodTemplateSpec](https://kubernetes.io/docs/api-reference/v1.9/#podtemplatespec-v1-core) alongside a ***graph*** which describes how components fit together.

In [12]:
!pygmentize resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                     

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [14]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [15]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:1 name:test-deployment-example-classifier-0 replicas:1] map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request

In [16]:
rest_request_ambassador("seldon-model",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "sa02nqhdt39637ihl5nptlvbdg",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.07746192278715774]
    }
  }
}


#### gRPC Request

In [17]:
 grpc_request_ambassador("seldon-model",API_AMBASSADOR)

meta {
  puid: "h8cg587824uf9bl9g1genvsf9p"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08034295501100448
  }
}



## Update deployment with canary

We will change the deployment to add a "canary" deployment. This illustrates:
 - Updating a deployment with no downtime
 - Adding an extra predictor to run alongside th exsting predictor.
 
 You could manage different traffic levels by controlling the number of replicas of each.

In [27]:
!pygmentize resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                     

In [28]:
!kubectl apply -f resources/model_with_canary.json -n seldon

seldondeployment "seldon-model" configured


Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [32]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-example-classifier-0 replicas:1 replicasAvailable:1] map[name:test-deployment-canary-svc-orch replicas:1 replicasAvailable:1] map[replicas:1 replicasAvailable:1 name:test-deployment-canary-mean-classifier-0]]]

#### REST Request

In [33]:
rest_request_ambassador("seldon-model",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "76563ebf7gisn122n1j0ldmoo2",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.08189241754225815, 0.08648345812157715]
    }
  }
}


#### gRPC request

In [34]:
 grpc_request_ambassador("seldon-model",API_AMBASSADOR)

meta {
  puid: "76f7vn7455300aca8k4nj92m9u"
}
data {
  names: "proba"
  tensor {
    shape: 2
    shape: 1
    values: 0.08050883671621968
    values: 0.08756216973724293
  }
}



## Load test

Start a load test which will post REST requests at 10 requests per second.

In [35]:
!helm install seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://seldon-core-seldon-apiserver:8080 \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --namespace seldon \
    --repo https://storage.googleapis.com/seldon-charts

NAME:   loadtest
LAST DEPLOYED: Fri Sep  7 19:54:53 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP     EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.105.91.100  <none>       5557:32266/TCP,5558:30861/TCP,8089:30446/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-qbx8s   0/1    ContainerCreating  0         0s
locust-master-1-c565k  0/1    ContainerCreating  0         0s




You should port-foward the grafana dashboard

```bash
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then iew an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:
 - Username : admin
 - password : password (as set when starting seldon-core-analytics above)
 
 The dashboard should look like below:
 
 
 <img src="images/dashboard.png" alt="predictor with canary" title="ml graph"/>

# Tear down

In [ ]:
!helm delete loadtest --purge

In [ ]:
!kubectl delete -f resources/model_with_canary.json -n seldon

In [ ]:
!helm delete seldon-core-analytics --purge

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge